In [1]:
# Spleeter separation (CPU-only) with museval evaluation
# This cell performs separation only and saves outputs under temp_estimates/spleeter_2stems/<subset>/
# and writes museval JSON outputs to temp_output/spleeter_2stems/ so the main notebook can compare models.
# It forces TensorFlow to use CPU by clearing CUDA_VISIBLE_DEVICES before importing Spleeter/TensorFlow.

import os
import numpy as np
import soundfile as sf
import musdb
import museval

# Force TensorFlow to use CPU (must be set before importing Spleeter/TensorFlow)
os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

from spleeter.separator import Separator

# Paths
path_to_folder = "."
musdb_root = os.path.join(path_to_folder, "musdb18")
estimates_base_path = os.path.join(path_to_folder, "estimates")
output_base_path = os.path.join(path_to_folder, "output")

model_name = "spleeter_2stems"
estimates_path = os.path.join(estimates_base_path, model_name)
output_path = os.path.join(output_base_path, model_name)

os.makedirs(estimates_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)

# Initialize Spleeter (2 stems: vocals + accompaniment)
separator = Separator("spleeter:2stems")

# Load MUSDB (main notebook handles download; here we avoid redownloading)
mus = musdb.DB(root=musdb_root, download=False)

for track in mus:
    print(f"[→] Separating: {track.name}")

    # Ensure audio is float32 to avoid TensorFlow dtype mismatches
    audio = track.audio.astype(np.float32)  # shape: (samples, channels)
    rate = track.rate
    subset = track.subset

    try:
        prediction = separator.separate(audio)
    except Exception as e:
        print(f"Spleeter separation failed for {track.name}: {e}")
        continue

    # prediction contains 'vocals' and 'accompaniment' as numpy arrays (samples, channels)
    estimates = {
        "vocals": np.asarray(prediction["vocals"], dtype=np.float32),
        "accompaniment": np.asarray(prediction["accompaniment"], dtype=np.float32),
    }

    # Prepare estimates for museval: museval expects numpy arrays shape (samples, channels)
    cpu_estimates = {key: arr for key, arr in estimates.items()}

    # Evaluate with museval and write JSON outputs into output_path
    try:
        scores = museval.eval_mus_track(track, cpu_estimates, output_dir=output_path)
        print(scores)
    except Exception as e:
        print(f"Museval evaluation failed for {track.name}: {e}")

    # Create subset folder and save wavs following the main notebook convention
    subset_path = os.path.join(estimates_path, subset)
    os.makedirs(subset_path, exist_ok=True)

    for target, audio_np in estimates.items():
        file_name = f"{track.name} - {target}.wav"
        out_path = os.path.join(subset_path, file_name)
        sf.write(out_path, audio_np, rate)

print("Spleeter separation complete.")

[→] Separating: A Classic Education - NightOwl
INFO:spleeter:Downloading model archive https://github.com/deezer/spleeter/releases/download/v1.4.0/2stems.tar.gz
INFO:spleeter:Validating archive checksum
INFO:spleeter:Extracting downloaded 2stems archive
INFO:spleeter:2stems model file(s) extracted
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Using config: {'_model_dir': 'pretrained_models/2stems', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.7
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def'